In [1]:
import pandas as pd
import pandasql as ps

# Корабли
Корабли в «классах» построены по одному и тому же проекту, и классу присваивается либо имя первого корабля, построенного по данному проекту, либо названию класса дается имя проекта, которое не совпадает ни с одним из кораблей в БД. Корабль, давший название классу, называется головным.
Отношение Classes содержит имя класса, тип (bb для боевого (линейного) корабля или bc для боевого крейсера), страну, в которой построен корабль, число главных орудий, калибр орудий (диаметр ствола орудия в дюймах) и водоизмещение ( вес в тоннах). В отношении Ships записаны название корабля, имя его класса и год спуска на воду. В отношение Battles включены название и дата битвы, в которой участвовали корабли, а в отношении Outcomes – результат участия данного корабля в битве (потоплен-sunk, поврежден - damaged или невредим - OK).
Замечания. 1) В отношение Outcomes могут входить корабли, отсутствующие в отношении Ships. 2) Потопленный корабль в последующих битвах участия не принимает.

![](ships.png)

In [2]:
classes = pd.read_csv('data/classes.csv', index_col='class')
classes.head()

,type,country,guns,bore,displacement
class,,,,,
Bismarck,bb,Germany,8,15.0,42000
Iowa,bb,USA,9,16.0,46000
Kongo,bc,Japan,8,14.0,32000
North Carolina,bb,USA,12,16.0,37000
Renown,bc,Gt.Britain,6,15.0,32000


In [3]:
ships = pd.read_csv('data/ships.csv', index_col='name')
ships.head()

,class,launched
name,,
California,Tennessee,1921
Haruna,Kongo,1916
Hiei,Kongo,1914
Iowa,Iowa,1943
Kirishima,Kongo,1915


In [4]:
outcomes = pd.read_csv('data/outcomes.csv', index_col=['ship', 'battle'])
outcomes.head()

,,result
ship,battle,
Bismarck,North Atlantic,sunk
California,Surigao Strait,OK
Duke of York,North Cape,OK
Fuso,Surigao Strait,sunk
Hood,North Atlantic,sunk


In [5]:
battles = pd.read_csv('data/battles.csv', index_col='name', parse_dates=['date'])
battles.head()

,date
name,
Guadalcanal,1942-11-15
North Atlantic,1941-05-25
North Cape,1943-12-26
Surigao Strait,1944-10-25


# Задача 1
По Вашингтонскому международному договору от начала 1922 г. запрещалось строить линейные корабли водоизмещением более 35 тыс.тонн. Укажите корабли, нарушившие этот договор (учитывать только корабли c известным годом спуска на воду). Вывести названия кораблей.

![](ships.png)

### SQL

In [6]:
query = '''
    select
        ships.name
    from
        ships inner join classes
        on ships.class = classes.class
    where 
        displacement > 35 and
        type = 'bb' and
        launched >= 1922 
    '''
ps.sqldf(query)

,name
0,Iowa
1,Missouri
2,Musashi
3,New Jersey
4,North Carolina
5,Washington
6,Wisconsin
7,Yamato
8,South Dakota


### Pandas

In [7]:
classes[
    (classes.type == 'bb') & (classes.displacement > 35)
].merge(
    ships[ships.launched >= 1922],
    left_index=True,
    right_on='class'
).reset_index()[['name']]

,name
0,Iowa
1,Missouri
2,New Jersey
3,Wisconsin
4,North Carolina
5,Washington
6,South Dakota
7,Musashi
8,Yamato


# Задача 2

Найдите корабли, "сохранившиеся для будущих сражений"; т.е. выведенные из строя в одной битве (damaged), они участвовали в другой, произошедшей позже. Здесь важно учесть дату.

![](ships.png)

### SQL

In [8]:
query = '''
    select t1.ship
    from (
        select
            ship, result, date
        from
            outcomes inner join battles
                on outcomes.battle = battles.name
    ) as t1 inner join (
        select
            ship, result, date
        from
            outcomes inner join battles
                on outcomes.battle = battles.name
    ) as t2
    on t1.ship=t2.ship and t1.date > t2.date and t2.result = 'damaged';
'''
ps.sqldf(query)

,ship
0,California


### Pandas

In [9]:
outcomes_battles = outcomes.reset_index().merge(
    battles, left_on='battle', right_index=True)[['ship', 'result', 'date']]
outcomes_battles

,ship,result,date
0,Bismarck,sunk,1941-05-25
4,Hood,sunk,1941-05-25
5,King George V,OK,1941-05-25
7,Prince of Wales,damaged,1941-05-25
8,Rodney,OK,1941-05-25
1,California,OK,1944-10-25
3,Fuso,sunk,1944-10-25
11,Tennessee,OK,1944-10-25
13,West Virginia,OK,1944-10-25
14,Yamashiro,sunk,1944-10-25


In [10]:
merge_result = outcomes_battles.merge(outcomes_battles, on='ship', suffixes=('_t1', '_t2'))
merge_result[(merge_result.date_t1 > merge_result.date_t2) & (merge_result.result_t2 == 'damaged')]

,ship,result_t1,date_t1,result_t2,date_t2
6,California,OK,1944-10-25,damaged,1942-11-15


# Задача 3
Укажите названия, водоизмещение и число орудий, кораблей участвовавших в сражении при Гвадалканале (Guadalcanal).
![](ships.png)

### SQL

In [11]:
query = '''
    select
        ships.name,
        classes.displacement,
        classes.guns
    from
        outcomes inner join ships
            on ships.name = outcomes.ship
        inner join classes
            on classes.class = ships.class
    where
        outcomes.battle = 'Guadalcanal'
'''
ps.sqldf(query)

,name,displacement,guns
0,Kirishima,32000,8
1,South Dakota,37000,12
2,Washington,37000,12
3,California,32000,12


### Pandas

In [12]:
merge_res = outcomes.reset_index().merge(
    ships, left_on='ship', right_index=True
).merge(
    classes, left_on='class', right_index=True
)
merge_res[merge_res.battle == 'Guadalcanal'][['ship', 'displacement', 'guns']]

,ship,displacement,guns
15,California,32000,12
6,Kirishima,32000,8
10,South Dakota,37000,12
12,Washington,37000,12
